<a href="https://colab.research.google.com/github/hrstbangera/NLP-with-Python/blob/master/finetune_falcon_7b_sharded_qna_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine-tune Falcon-7b-instruct-sharded model** on a mental health conversational dataset curated by heliosbrahma can be found on Hugging Face.
Links to both the model and dataset are in the notebook.


##Installs and imports

In [2]:
#all installs
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install huggingface_hub

#all imports
import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

##Notebook connection to Hugging face

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
notebook_login()

##Loading the dataset from hugging face

In [4]:
# dataset_name = "heliosbrahma/mental_health_chatbot_dataset"
# data = load_dataset(dataset_name)
# data

In [5]:
# train_data = data["train"]

In [6]:
# import pandas as pd

# # Create an empty list to store the text data
# text_data = []

# # Iterate through the "train" split and append text values to the list
# for row in train_data:
#     text_data.append(row["text"])

# # Create the DataFrame directly from the list
# df = pd.DataFrame({"text": text_data})

# # Print the DataFrame
# print(df.head())

In [7]:
# print(df['text'][1])

In [10]:
import pandas as pd
df = pd.read_csv("output.csv")

In [11]:
df.head(1)

,Context,Question,Answer
0,\n \nAIRCRAFT \nINSURANCE \nBINDER \n \n \n...,What is the combined single limit each occurre...,"$200,000,000."


In [12]:
df['Context_length'] = df['Context'].str.len()

# Display the DataFrame
print(df)

                                               Context  \
0     \n \nAIRCRAFT  \nINSURANCE \nBINDER \n \n  \n...   
1     \n \nAIRCRAFT  \nINSURANCE \nBINDER \n \n  \n...   
2     \n \nAIRCRAFT  \nINSURANCE \nBINDER \n \n  \n...   
3     \n \nAIRCRAFT  \nINSURANCE \nBINDER \n \n  \n...   
4     \n \nAIRCRAFT  \nINSURANCE \nBINDER \n \n  \n...   
..                                                 ...   
421  Policy No: 13000294 Page 1Policy No:\nBind Dat...   
422  Policy No: 13000294 Page 1Policy No:\nBind Dat...   
423  Policy No: 13000294 Page 1Policy No:\nBind Dat...   
424  Policy No: 13000294 Page 1Policy No:\nBind Dat...   
425  Policy No: 13000294 Page 1Policy No:\nBind Dat...   

                                              Question  \
0    What is the combined single limit each occurre...   
1    What is the total annual premium for all appli...   
2    Who is the Director of Aviation Safety Program...   
3    What is the coverage limit for Non-Owned Aircr...   
4    What is 

In [15]:
df['Context_length'].nunique()

43

In [ ]:
import pandas as pd
import torch
# from transformers import AutoTokenizer
# from your_custom_trainer_module import SFTTrainer

# Load your CSV
# df = pd.read_csv("your_dataset.csv")

# Preprocess and combine your columns
df["text"] = df.apply(lambda row: f"context: {row.context} question: {row.question}", axis=1)
df["completion"] = df["answer"]

# Define a custom dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# Tokenize
tokenizer = AutoTokenizer.from_pretrained("your-model-name")
encodings = tokenizer(df["text"].tolist(), df["completion"].tolist(), truncation=True, padding=True, max_length=max_seq_length)

# Create dataset
dataset = CustomDataset(encodings)

# Initialize trainer
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Start training
trainer.train()


##Loading the model and Setting up bitsandbytes config

We will use sharded version of falcon-7b-instruct model



In [16]:
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00013-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/828M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

##Loading the tokenizer

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

##Setting up the LoRA config

In [18]:
model_peft = prepare_model_for_kbit_training(model)

lora_alpha = 32 #16
lora_dropout = 0.05 #0.1
lora_rank = 32 #64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model_peft, peft_config)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


##Load the trainer

In [19]:
output_dir = "falcon7binstruct_mentalhealthmodel_jan23"
per_device_train_batch_size = 16 #4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100 #100 #500
warmup_ratio = 0.03
lr_scheduler_type = "cosine" #"constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True
)

In [9]:
import pandas as pd
from datasets import Dataset
from transformers import TrainingArguments, Trainer
from torch.utils.data import Dataset

# 1. Load the CSV Data
data = pd.read_csv("output.csv")

# 2. Preprocess the Data
# Assuming the model takes input in the format "context: [context] question: [question]"
data['text'] = "context: " + data['Context'] + " question: " + data['Question'] + " answer: " + data['Answer']



In [24]:
# data['text'][1]

In [25]:
# 3. Create a Dataset
class CustomDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(text, max_length=self.max_length, padding='max_length', truncation=True)
        return inputs

In [27]:
!pip install datasets

In [28]:
import pandas as pd
from datasets import Dataset
from transformers import TrainingArguments, Trainer
import torch
from torch.utils.data import Dataset as TorchDataset

In [29]:
max_seq_length = 256
# Convert DataFrame to HuggingFace Dataset
hf_dataset = Dataset.from_pandas(data)
train_dataset = CustomDataset(hf_dataset['text'], tokenizer, max_seq_length)

# 4. Adjust the Trainer
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_dataset,  # Use the custom dataset
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Proceed with training as before
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,2.619600
20,1.309300
30,0.473200
40,0.111400
50,0.040700
60,0.028900
70,0.022300
80,0.019600
90,0.017700
100,0.017400


TrainOutput(global_step=100, training_loss=0.46601767614483836, metrics={'train_runtime': 5128.6216, 'train_samples_per_second': 1.248, 'train_steps_per_second': 0.019, 'total_flos': 6.49170049278935e+16, 'train_loss': 0.46601767614483836, 'epoch': 14.81})

In [ ]:
falcon7binstruct_mentalhealthmodel_jan23

In [12]:
context1 = df['Context'][1]

In [13]:
question1 = df['Question'][1]

In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Load the Fine-Tuned Model
# Replace 'path_to_fine_tuned_model' with the actual path to your fine-tuned model
model_path = "falcon7binstruct_mentalhealthmodel_jan23"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Ensure the model is in evaluation mode
model.eval()

# Function for inference
def generate_answer(context, question):
    input_text = f"context: {context} question: {question}"
    inputs = tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True)

    # Generate a response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512, num_beams=5, early_stopping=True)

    # Decode the generated text
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Example usage
context1 = "Your example context here."
question1 = "Your example question here."
answer = generate_answer(context1, question1)
print(answer)


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


KeyboardInterrupt: 

##Passing arguments to the SFTT trainer

In [ ]:
max_seq_length = 256

trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

##Train the model

You can check your training time if you are doing multiple experiments

In [ ]:
start = time.time()

In [ ]:
peft_model.config.use_cache = False
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.656800
20,1.301300
30,0.912100
40,0.459900
50,0.170700
60,0.066200
70,0.045700
80,0.040500
90,0.038800
100,0.038000


TrainOutput(global_step=100, training_loss=0.4729997545480728, metrics={'train_runtime': 660.6616, 'train_samples_per_second': 9.687, 'train_steps_per_second': 0.151, 'total_flos': 5.302835892765082e+16, 'train_loss': 0.4729997545480728, 'epoch': 36.36})

In [ ]:
end=time.time()

In [ ]:
time_taken=end-start
print(time_taken)

675.8279531002045


##Save the model

In [ ]:
#trainer.save() #if you want to save your model locally

##Push to hub

In [30]:
trainer.push_to_hub()

events.out.tfevents.1706529801.df4e295961a5.3911.0:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HarshithNLP/falcon7binstruct_mentalhealthmodel_jan23/commit/a7a8ffb0e4abef4502efbc6cedcc3849ca62f49a', commit_message='End of training', commit_description='', oid='a7a8ffb0e4abef4502efbc6cedcc3849ca62f49a', pr_url=None, pr_revision=None, pr_num=None)

##Inference

In [ ]:
# full version model
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [40]:
# Loading PEFT model
PEFT_MODEL = "HarshithNLP/falcon7binstruct_mentalhealthmodel_jan23"
config = PeftConfig.from_pretrained(PEFT_MODEL,load_in_8bit_fp32_cpu_offload=True)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="cpu",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [41]:
# Generate responses from both orignal model and fine-tuned model
def get_response(context,question):
  prompt = f"""
  ###Instruction: you are provided with context and question. You need to generate a response.
  ###Context: {context}
  ###Question: {question}

  ###Response:

  """

  # encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
  # outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
  #                                                                                                                    eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
  #                                                                                                                    temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  # text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  # #print(dashline)
  # print(f'Response from original falcon_7b_instruct_sharded:\n{text_output}')

  # print("*******************************************************")

  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to("cuda:0")
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'Response from fine-tuned falcon_7b_instruct_sharded:\n{peft_text_output}')


In [42]:
get_response(context1,question1)

Token indices sequence length is longer than the specified maximum sequence length for this model (7358 > 2048). Running this sequence through the model will result in indexing errors


RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'

In [5]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
load_in_8bit_fp32_cpu_offload=True
peft_model_id = "HarshithNLP/falcon7binstruct_mentalhealthmodel_jan23"
config = PeftConfig.from_pretrained(peft_model_id,load_in_8bit_fp32_cpu_offload=True)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,return_dict=True, load_in_8bit=False, device_map='cpu')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


adapter_model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [39]:
import torch

# Ensure the prompt is in the correct format
import torch
from torch.cuda.amp import autocast

# Ensure the prompt is in the correct format
# prompt = """A BS/MS in Computer Science or related field Preferred : 2+ years of programming experience writing code in Java, C++, C#, or C\n\n###\n\n"""
# prompt="""A BS/MS in Computer Science or related field Preferred : 2+ years of programming experience writing code in Java, C++, C#, or C. \n\n###\n\n"""
prompt = f"""
  ###Instruction: you are provided with context and question. You need to generate a response.
  ###Context: {context2}
  ###Question: {question2}

  ###Response:

  """

# Tokenize the prompt
input_ids = tokenizer(prompt, return_tensors='pt').input_ids

# Select device based on availability and memory considerations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_ids = input_ids.to(device)

try:
    # Generate the output with memory efficiency in mind
    with autocast(enabled=device.type == "cuda"):  # Use autocast only if on GPU
        output = model.generate(input_ids=input_ids, max_length=50, num_return_sequences=1)

    # Decode the generated tokens to a string
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(generated_text)

except RuntimeError as e:
    print("RuntimeError:", e)
    if device.type == "cuda":
        torch.cuda.empty_cache()  # Clear unused tensors to free up GPU memory

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



  ###Instruction: you are provided with context and question. You need to generate a response.
  ###Context: ALL OTHER TERMS AND CONDITIONS REMAIN UNCHANGED.
This endorsement is effective: May 29, 2018 Endorsement Premium: $27,220
Attached to and made part of Policy No.: 60000064
Issued to: ASTOR STREET ASSET MANAGEMENT, INC.
Global Aerospace, Inc.
BY: 
BHG001 (April 1, 2015)Endorsement No. 36
Page 1 of 1Policy Changes
The Policy is amended as follows:
The following items(s):
Branch Office Broker
The Companies Named Insured
Address of Named Insured XPolicy Period
Limits of the Company's Liability XDescription of Insured Aircraft
Aircraft Use Pilots
Premium
The premium for the portion of the policy period from May 29, 2018 to May 29, 2019 is $27,220.
It is agreed that the description of the aircraft shown in Item 4 of the Declarations is amended to read as follows:
Deductibles: Seats:
Year, Make and ModelIdent.
Number Insured Value In MotionNot
In MotionCrew
Member Passenger
2006 GULFS

In [16]:
context1

' \n \nAIRCRAFT  \nINSURANCE \nBINDER \n \n  \n \n  \n \n  \n \n  \n \n  \n \n \n \nPolicyholder Emerson Electric Co., et al \nAddress 8000 West Florissant Avenue \n St. Louis, Missouri 63136 Policy Term November 1, 2021 - 2022 \n \n   \n3:51 PM  10/13/2021 Page 2 of 14  \n \n \n \n \nCompanies A.M. Best Ratings (as of January 1, 2020) \n \n\uf084 National Liability & Fire Insurance Company A++ \n\uf084 ACE American Insurance Company A++ \n \n \nSome or all of the above Member Compan ies may appear on your actual policy. \n \n  \n \n \nThank you for giving USAIG the opportunity to partner with you for your aviation insurance needs. We know you have many \nchoices, but we’re confident that our programs and responsi ve, customer-focused services will show why we’ve been an \nindustry leader for more than eight decades. We make it our mission to exceed your expectations, providing world-class aviation insurance products and services while delivering exceptional value. The USAIG team is av

In [17]:
question1

'What is the total annual premium for all applicable aircraft under the USAIG policy SIHL1-G550?'

In [18]:
df

,Context,Question,Answer
0,\n \nAIRCRAFT \nINSURANCE \nBINDER \n \n \n...,What is the combined single limit each occurre...,"$200,000,000."
1,\n \nAIRCRAFT \nINSURANCE \nBINDER \n \n \n...,What is the total annual premium for all appli...,"$176,293."
2,\n \nAIRCRAFT \nINSURANCE \nBINDER \n \n \n...,Who is the Director of Aviation Safety Program...,Paul Ratté; by emailing safety@usaig.com.
3,\n \nAIRCRAFT \nINSURANCE \nBINDER \n \n \n...,What is the coverage limit for Non-Owned Aircr...,"$250,000,000."
4,\n \nAIRCRAFT \nINSURANCE \nBINDER \n \n \n...,What is the deductible for Garagekeeper's Liab...,"$1,000."
...,...,...,...
421,Policy No: 13000294 Page 1Policy No:\nBind Dat...,What are the endorsements listed on the policy?,"['E020 Additional Exclusions', 'E021 Rotorcraf..."
422,Policy No: 13000294 Page 1Policy No:\nBind Dat...,What is the hull value for the aircraft with F...,"$1,250,000"
423,Policy No: 13000294 Page 1Policy No:\nBind Dat...,What is the liability limit for the aircraft w...,"$50,000,000"
424,Policy No: 13000294 Page 1Policy No:\nBind Dat...,What is the commission percentage for the airc...,15.00%


In [19]:
df['Context_length'] = df['Context'].str.len()

In [28]:
df['Context_length'].min()

1062

In [29]:
min_index = df['Context_length'].idxmin()

# Then, use this index to select the row
min_row = df.loc[min_index]


In [32]:
min_row

Context           ALL OTHER TERMS AND CONDITIONS REMAIN UNCHANGE...
Question          What is the endorsement premium for the amendm...
Answer                                                     $27,220.
Context_length                                                 1062
Name: 133, dtype: object

In [31]:
min_index

133

In [37]:
context2 = df['Context'][min_index]

In [34]:
question2 = df['Question'][min_index]

In [35]:
question2

'What is the endorsement premium for the amendment made to Policy No. 60000064 on May 29, 2018?'

In [38]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import PeftModel, PeftConfig

peft_model_id = "HarshithNLP/falcon7binstruct_mentalhealthmodel_jan23"
config = PeftConfig.from_pretrained(peft_model_id, load_in_8bit_fp32_cpu_offload=True)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Ensuring the pad token is set for the tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False, device_map='cpu')
model = PeftModel.from_pretrained(model, peft_model_id)

# Define your context and question here
# context1 = "Your context here"
# question1 = "Your question here"

prompt = f"""
###Instruction: you are provided with context and question. You need to generate a response.
###Context: {context2}
###Question: {question2}

###Response:
"""

input_ids = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=2048).input_ids
attention_mask = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=2048).attention_mask

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

try:
    with torch.cuda.amp.autocast(enabled=device.type == "cuda"):
        output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50, num_return_sequences=1)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(generated_text)

except RuntimeError as e:
    print("RuntimeError:", e)
    if device.type == "cuda":
        torch.cuda.empty_cache()



Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



###Instruction: you are provided with context and question. You need to generate a response.
###Context: ALL OTHER TERMS AND CONDITIONS REMAIN UNCHANGED.
This endorsement is effective: May 29, 2018 Endorsement Premium: $27,220
Attached to and made part of Policy No.: 60000064
Issued to: ASTOR STREET ASSET MANAGEMENT, INC.
Global Aerospace, Inc.
BY: 
BHG001 (April 1, 2015)Endorsement No. 36
Page 1 of 1Policy Changes
The Policy is amended as follows:
The following items(s):
Branch Office Broker
The Companies Named Insured
Address of Named Insured XPolicy Period
Limits of the Company's Liability XDescription of Insured Aircraft
Aircraft Use Pilots
Premium
The premium for the portion of the policy period from May 29, 2018 to May 29, 2019 is $27,220.
It is agreed that the description of the aircraft shown in Item 4 of the Declarations is amended to read as follows:
Deductibles: Seats:
Year, Make and ModelIdent.
Number Insured Value In MotionNot
In MotionCrew
Member Passenger
2006 GULFSTREA

In [ ]:
get_response("What’s the difference between psychotherapy and counselling?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: What’s the difference between psychotherapy and counselling?

  ###Response:

  ###Therapy is a broad term that can refer to a range of different approaches to mental health treatment. It can include a variety of different techniques, such as cognitive behavioral therapy, psychodynamic therapy, and mindfulness-based therapies. Counseling, on the other hand, is a more specific term that generally refers to a shorter-term, goal-oriented approach that focuses on helping individuals achieve specific goals or improve their relationships. It may involve techniques from both therapy and counseling, depending on the individual's needs.
*******************************************************
Response from fine-tuned falcon_7b_instruct_sharded:

In [ ]:
get_response("What happens in a therapy session?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: What happens in a therapy session?

  ###Response:

  - A therapist will ask questions to understand your thoughts and feelings.
  - They may also use techniques to help you process your emotions and develop coping strategies.
  - The goal of therapy is to help you identify and change negative patterns of thinking and behavior.
  - The therapist may also provide homework assignments to practice what you learn in therapy.
  - The frequency and duration of therapy sessions can vary depending on the individual and their needs.
*******************************************************
Response from fine-tuned falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If yo

In [ ]:
get_response("Are neurofeedback and biofeedback the same thing?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: Are neurofeedback and biofeedback the same thing?

  ###Response:

  ###Explanation:
  Neurofeedback and biofeedback are similar in that they both involve the use of technology to help individuals learn how to control their body and mind. However, there are some differences between the two. Neurofeedback is typically used to help individuals learn how to control their brain waves, while biofeedback is used to learn how to control other bodily functions, such as heart rate or blood pressure. Additionally, neurofeedback is often used to treat conditions such as anxiety, depression, and ADHD, while biofeedback is often used to treat conditions such as chronic pain and migraines.
*******************************************************
Res

In [ ]:
get_response(" What is insomnia disorder?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question:  What is insomnia disorder?

  ###Response:

  ###Explanation: Insomnia disorder is a sleep disorder where a person has difficulty falling asleep or staying asleep. It can be caused by a variety of factors, including anxiety, depression, and certain medications. It can also lead to other health problems if left untreated. It is important to seek medical attention if you suspect you have Insomnia disorder.

Incorrect response: 'Insomnia disorder is a sleep disorder where a person has difficulty falling asleep or staying asleep.'

Correct response: 'Insomnia disorder is a sleep disorder where a person has difficulty falling asleep or staying asleep. It can be caused by a variety of factors, including anxiety, depression, and certain med